In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib.pyplot as plt

2023-04-24 16:07:37.861128: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 16:07:38.023485: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-24 16:07:38.794921: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/deadpool/anaconda3/lib/:/home/deadpool/anaconda3/lib/:/home/deadpool/anaconda3/lib/
2023-04-24 16:07:38.795016: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic librar

In [ ]:
# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Define image parameters
img_size = 64
img_shape = (img_size, img_size, 1)
batch_size = 64

## Generator

In [ ]:
# # 64x64 
def generator_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(8*8*128, input_shape=(128,), use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Reshape((8, 8, 128)),
        tf.keras.layers.Conv2DTranspose(128, (5,5), strides=(2,2), padding='same', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', use_bias=False, activation='tanh')
    ])
    return model
generator = generator_model()
generator.summary()

In [ ]:
# def generator_model():
#     model = tf.keras.Sequential([
#         tf.keras.layers.Dense(16 * 16 * 64, input_shape=(128,), use_bias=False),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.LeakyReLU(),
#         tf.keras.layers.Reshape((16, 16, 64)),
#         tf.keras.layers.Conv2DTranspose(512, (4,4), strides=(2,2), padding='same', use_bias=False),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.LeakyReLU(),
#         tf.keras.layers.Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', use_bias=False),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.LeakyReLU(),
#         tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', use_bias=False),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.LeakyReLU(),
#         tf.keras.layers.Conv2DTranspose(64, (4,4), strides=(1,1), padding='same', use_bias=False),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.LeakyReLU(),
#         tf.keras.layers.Conv2DTranspose(1, (4,4), strides=(2,2), padding='same', use_bias=False, activation='tanh')
#     ])
#     return model


## Discriminator

In [ ]:
## 64x64
def discriminator_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (5,5), strides=(2,2), padding='same', input_shape=img_shape),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(128, (5,5), strides=(2,2), padding='same'),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
generator = generator_model()
generator.summary()

In [ ]:
discriminator = discriminator_model()
discriminator.summary()

## Loss Functions

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(4e-4)

### Checkout points to check progress

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

### Function to generate and save images

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(8, 8))
    for i in range(predictions.shape[0]):
        plt.subplot(8, 8, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')
    plt.show()

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, 128])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
        disc_loss_real = cross_entropy(tf.ones_like(real_output), real_output)
        disc_loss_fake = cross_entropy(tf.zeros_like(fake_output), fake_output)
        disc_loss = disc_loss_real + disc_loss_fake

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

### Load and preprocess data

In [ ]:
# Load and preprocess the data
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=1, dtype=tf.uint8)
    image = tf.image.resize(image, [img_size, img_size])
    image = tf.cast(image, tf.float32)  # Convert the data type to float32
    image = (image - 127.5) / 127.5  # Normalize the images to [-1, 1]
    return image

list_ds = tf.data.Dataset.list_files('dataset_1/*.png')
dataset = list_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(batch_size)

## Train Model

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        for batch in dataset:
            train_step(batch)

        # Save the model every 60 epochs
        if (epoch + 1) % 10 == 0:
            test_input = tf.random.normal([64, 128])
            generate_and_save_images(generator, epoch+1, test_input)
            checkpoint.save(file_prefix=checkpoint_prefix)

        print('Epoch {} complete'.format(epoch+1))

# Train the models
train(dataset, epochs=600)


In [ ]:
# def on_epoch_end():
#     random_latent_vectors = tf.random.normal(shape=(5, 128))
#     generated_images = generator(random_latent_vectors)
#     generated_images *= 255
#     generated_images.numpy()
#     for i in range(5):
#         img = keras.preprocessing.image.array_to_img(generated_images[i])
#         img.save('/home/deadpool/Honors-Project/Honors-Project/GAN_generated_images/final_images/64x64_generated_d1_%d_900e.png' % i)

# on_epoch_end()

In [ ]:
generator.compile()
generator.save('generator256x256_d1_1000.h5')

In [2]:
loaded_model = tf.keras.models.load_model('./generators/generator64x64_1500.h5')

2023-04-24 16:07:42.663701: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 16:07:42.670392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 16:07:42.670733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-24 16:07:42.671361: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [ ]:
# def generate_images():
#     random_latent_vectors = tf.random.normal(shape=(5, 100))
#     generated_images = loaded_model(random_latent_vectors)
#     generated_images *= 255
#     generated_images.numpy()
#     for i in range(5):
#         img = keras.preprocessing.image.array_to_img(generated_images[i])
#         img.save('/home/deadpool/Honors-Project/Honors-Project/GAN_generated_images/final_images/generated_%d_from_saved_d1_1500e.png' % i)

# generate_images()

In [4]:
num_samples = 1800
def generate_images():
    random_latent_vectors = tf.random.normal(shape=(num_samples, 100))
    generated_images = loaded_model(random_latent_vectors)
    generated_images *= 255
    generated_images = generated_images.numpy()
    for i in range(num_samples):
        img = keras.preprocessing.image.array_to_img(generated_images[i])
        img.save('./GAN_generated_images/test1/img%d.png' % i)
    images = []
    for i in range(1800):
        img = generated_images[i].astype(np.uint8)
        images.append(img)
    return images

images = generate_images()

# fig, axes = plt.subplots(nrows=10, ncols=10, figsize=(8,8))
# axes = axes.flatten()
# for img, ax in zip(images, axes):
#     ax.imshow(img, cmap='gray')
#     ax.axis('off')
# plt.tight_layout()
# plt.savefig('./GAN_generated_images/Plot/256x256_figure.png')
# plt.show()

2023-04-24 16:08:34.015313: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-04-24 16:08:34.900488: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-24 16:08:34.901339: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-24 16:08:34.901358: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-04-24 16:08:34.902345: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-24 16:08:34.902429: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-04-24 16:08:36.320156: W tensorflow/core/common_runtime/bf

In [ ]:
# Generate images and plt and test performance (Speed)
import time
import numpy as np
import tensorflow as tf

num_samples = 10
images = []

def generate_images():
    random_latent_vectors = tf.random.normal(shape=(num_samples, 128))
    generated_images = loaded_model(random_latent_vectors)
    generated_images *= 255
    generated_images = generated_images.numpy()
    for i in range(num_samples):
        img = generated_images[i].astype(np.uint8)
        images.append(img)
    return images

# Load the pre-trained DCGAN model
loaded_model = tf.keras.models.load_model('./generators/generator64x64_d1_700.h5')

# Time the generation of one image
start_time = time.time()
_ = generate_images()
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to generate {num_samples} images: {elapsed_time:.5f} seconds")


fig, axes = plt.subplots(nrows=10, ncols=10, figsize=(8,8))
axes = axes.flatten()
for img, ax in zip(images, axes):
    ax.imshow(img, cmap='gray')
    ax.axis('off')
plt.tight_layout()
plt.show()